In [ ]:
# GOOGLE COLAB: YOLO11 NANO TRAINING
# Real-Time Object Detection - Milestone 2

# CELL 1: Setup Environment

# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"CUDA Version: {torch.version.cuda}")

# Install required packages
!pip install ultralytics azure-storage-blob azure-identity

from ultralytics import YOLO
import os

GPU Available: False
GPU Name: None
CUDA Version: 12.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.0/429.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.3/191.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# 2: Download Data from Azure Storage

!pip install -q azure-storage-blob

from azure.storage.blob import BlobServiceClient
from tqdm import tqdm

# Azure Storage credentials (UPDATE THESE!)
STORAGE_ACCOUNT_NAME = "depiproject"
STORAGE_ACCOUNT_KEY = "c7m2YGOz+d8GZASdHqzNTbiv4WiZkPzH9YlsCVycsN7nWTsiJv646j3B96fWcL5rPSyTXVtfjNiZ+AStICSi5Q=="  # Get from Azure Portal
CONTAINER_NAME = "azureml-blobstore-7396e1aa-882f-4d22-a05d-6bd0bf3ea95c"  # Default ML workspace container

# Connection string
connection_string = f"DefaultEndpointsProtocol=https;AccountName={STORAGE_ACCOUNT_NAME};AccountKey={STORAGE_ACCOUNT_KEY};EndpointSuffix=core.windows.net"

# Connect
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

# Download function
def download_from_azure(blob_prefix, local_path):
    os.makedirs(local_path, exist_ok=True)
    blobs = list(container_client.list_blobs(name_starts_with=blob_prefix))

    for blob in tqdm(blobs, desc=f"Downloading {blob_prefix}"):
        blob_client = container_client.get_blob_client(blob.name)
        local_file = os.path.join(local_path, os.path.basename(blob.name))

        with open(local_file, "wb") as f:
            f.write(blob_client.download_blob().readall())

    print(f" Downloaded {len(blobs)} files to {local_path}")

# Download KITTI dataset
print("Downloading KITTI from Azure...")
download_from_azure('kitti/training/image_2/', '/content/kitti/images/')
download_from_azure('kitti/training/label_2/', '/content/kitti/labels/')

print("Dataset ready in Colab!")

 Downloaded 7481 files to /content/kitti/images/


 Downloaded 7481 files to /content/kitti/labels/
Dataset ready in Colab!


In [ ]:
# 3: Convert KITTI to YOLO Format

import os
import shutil

# KITTI to YOLO class mapping
KITTI_TO_YOLO = {
    'Car': 0,
    'Van': 1,
    'Truck': 2,
    'Pedestrian': 3,
    'Person_sitting': 4,
    'Cyclist': 5,
    'Tram': 6,
    'Misc': 7
}

def kitti_to_yolo(kitti_line, img_width, img_height):
    """Convert KITTI annotation to YOLO format"""
    parts = kitti_line.strip().split()
    if len(parts) < 15:
        return None

    obj_type = parts[0]
    if obj_type not in KITTI_TO_YOLO or obj_type == 'DontCare':
        return None

    class_id = KITTI_TO_YOLO[obj_type]

    # KITTI bbox: left, top, right, bottom
    left, top, right, bottom = map(float, parts[4:8])

    # Convert to YOLO: x_center, y_center, width, height (normalized)
    x_center = ((left + right) / 2) / img_width
    y_center = ((top + bottom) / 2) / img_height
    width = (right - left) / img_width
    height = (bottom - top) / img_height

    return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"

# Convert all KITTI labels to YOLO format
from PIL import Image

images_dir = '/content/kitti/images'
labels_dir = '/content/kitti/labels'
yolo_dir = '/content/kitti_yolo'

os.makedirs(f'{yolo_dir}/images/train', exist_ok=True)
os.makedirs(f'{yolo_dir}/labels/train', exist_ok=True)

image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]

for img_file in tqdm(image_files, desc="Converting to YOLO format"):
    # Get image dimensions
    img_path = os.path.join(images_dir, img_file)
    img = Image.open(img_path)
    img_width, img_height = img.size

    # Copy image
    shutil.copy(img_path, f'{yolo_dir}/images/train/{img_file}')

    # Convert label
    label_file = img_file.replace('.png', '.txt')
    label_path = os.path.join(labels_dir, label_file)

    if os.path.exists(label_path):
        yolo_labels = []
        with open(label_path, 'r') as f:
            for line in f:
                yolo_line = kitti_to_yolo(line, img_width, img_height)
                if yolo_line:
                    yolo_labels.append(yolo_line)

        # Save YOLO label
        if yolo_labels:
            yolo_label_path = f'{yolo_dir}/labels/train/{label_file}'
            with open(yolo_label_path, 'w') as f:
                f.writelines(yolo_labels)

print(" KITTI converted to YOLO format!")

Converting to YOLO format: 100%|██████████| 7481/7481 [01:11<00:00, 104.21it/s]

 KITTI converted to YOLO format!


In [ ]:
# 4: Create YOLO Dataset Configuration

dataset_yaml = f"""
# KITTI Dataset for YOLO
path: /content/kitti_yolo
train: images/train
val: images/train  # Using same for now, will split later

# Classes
nc: 8
names: ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc']
"""

with open('/content/kitti_yolo/data.yaml', 'w') as f:
    f.write(dataset_yaml)

print(" Dataset configuration created!")

 Dataset configuration created!


In [ ]:
# 5: Train YOLO11 Nano

# Initialize YOLO11 Nano model
model = YOLO("/content/yolo11n.pt")  # Nano version - smallest and fastest

# Train
results = model.train(
    data='/content/kitti_yolo/data.yaml',
    epochs=70,
    imgsz=640,
    batch=16,
    device=0,  # GPU
    project='yolo11_kitti',
    name='yolo11n_100epochs',
    patience=20,
    save=True,
    plots=True
)

print("Training complete!")

Ultralytics 8.3.236 🚀 Python-3.12.12 torch-2.9.0+cu126 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
# 6: Evaluate Model

# Validate model
metrics = model.val()

print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")

# Test on sample images
results = model.predict(source='/content/kitti_yolo/images/train', save=True)

print(" Evaluation complete!")

In [ ]:
# 7: Upload Trained Model to Azure

# Export model
model.export(format='onnx')

# Upload to Azure Storage
trained_model_path = 'yolo11_kitti/yolo11n_100epochs/weights/best.pt'
onnx_model_path = 'yolo11_kitti/yolo11n_100epochs/weights/best.onnx'

# Upload function
def upload_to_azure(local_file, blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    with open(local_file, 'rb') as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f" Uploaded: {blob_name}")

# Upload models
upload_to_azure(trained_model_path, 'models/yolo11n_best.pt')
upload_to_azure(onnx_model_path, 'models/yolo11n_best.onnx')

# Upload training results
import shutil
shutil.make_archive('/content/training_results', 'zip', 'yolo11_kitti/yolo11n_100epochs')
upload_to_azure('/content/training_results.zip', 'models/training_results.zip')

print("Models uploaded to Azure Storage!")